# Tarea 2

### Cuerpo Docentes

- Profesores: [Andrés Abeliuk](https://aabeliuk.github.io/), [Fabián Villena](https://villena.cl/).
- Profesor Auxiliar: Martín Paredes

### Instrucciones generales

- Grupos de máximo 4 personas.
- Esta prohibido compartir las respuestas con otros grupos.
- Indicios de copia serán penalizados con la nota mínima.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente, si utiliza material extra debe citarlo.


### Integrantes

- Fabio Bórquez
- Luis Berríos
- Juan Román
- Marcos Molina

## Contexto

El discurso de odio es cualquier expresión que promueva o incite a la discriminación, la hostilidad o la violencia hacia una persona o grupo de personas en una relación asimétrica de poder, tal como la raza, la etnia, el género, la orientación sexual, la religión, la nacionalidad, una discapacidad u otra característica similar.

En cambio, la incivilidad se refiere a cualquier comportamiento o actitud que rompe las normas de respeto, cortesía y consideración en la interacción entre personas. Esta puede manifestarse de diversas formas, tal como insultos, ataques personales, sarcasmo, desprecio, entre otras.

En esta tarea tendrán a su disposición un dataset de textos con las etiquetas `odio`, `incivilidad` o `normal`. La mayor parte de los datos se encuentra en español de Chile. Con estos datos, deberán entrenar un modelo que sea capaz de predecir la etiqueta de un texto dado.

El corpus para esta tarea se compone de 3 datasets:  
- [Multilingual Resources for Offensive Language Detection de Arango et al. (2022)](https://aclanthology.org/2022.woah-1.pdf#page=136)
- [Dataton UTFSM No To Hate (2022)](http://dataton.inf.utfsm.cl/)
- Datos generados usando la [API de GPT3 (modelo DaVinci 03)](https://platform.openai.com/docs/models/gpt-3).

Agradecimientos a los autores por compartir los datos y a David Miranda, Fabián Diaz, Santiago Maass y Jorge Ortiz por revisar y reetiquetar los datos en el contexto del curso "Taller de Desarrollo de Proyectos de IA" (CC6409), Departamento de Ciencias de la Computación, Universidad de Chile.

Los datos solo pueden ser usados con fines de investigación y docencia. Está prohibida la difusión externa.


## Tarea a resolver

Para esta tarea 2, buscaremos desarrollar un *benchmark* sobre una tarea de clasificación de NLP. Un benchmark es básicamente utilizar diferentes técnicas para resolver una misma tarea específica, en este caso seguiremos buscando alternativas para resolver el problema de clasificación de la tarea 1. Particularmente, se le pide:

- Implementar una arquitectura en RNN utilizando PyTorch.
- Utilizar transformers para revolver el problema de clasificación, en especifico utilizar BETO.
- Utilizar algún LLM utilizando Zero y Few short learning para resolver el problema de clasificación.


### Cargar el dataset


En esta sección, cargaremos el dataset desde el repositorio del módulo. Para ello ejecute las siguientes líneas:

In [1]:
import pandas as pd

In [2]:
# Dataset.
dataset_df = pd.read_csv("https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/new/assignment_1/train/train.tsv", sep="\t")


### Analizar los datos

En esta sección analizaremos el balance de los datos. Para ello se imprime la cantidad de tweets de cada dataset agrupados por la intensidad de sentimiento.

In [3]:
dataset_df.sample(5)

,id,texto,clase
11188,7896,@user Sabrá acaso este actor de pacotilla que ...,incivilidad
11073,1306,Se podrían aislar las facultades en área Salud...,normal
4097,3001,@user Estos negros son un cacho,incivilidad
1888,12875,"La meta es: una sola mujer, varios negocios y ...",normal
346,11885,@user @user @user @user @user @user Chilw está...,odio


In [4]:
dataset_df["clase"].value_counts()

,count
clase,
incivilidad,5424
normal,4280
odio,2510


In [5]:
target_classes = list(dataset_df['clase'].unique())
target_classes

['normal', 'incivilidad', 'odio']

### Instalar librerias

Puede usar esta celda para instalar las librerías que estime necesario.

In [6]:
%%capture



### Importar librerías

En esta sección, importamos la liberías necesarias para el correcto desarrollo de esta tarea. Puede utilizar otras librerías que no se en encuentran aquí, pero debe citar su fuente.

In [10]:
!pip uninstall -y torch torchvision torchaudio

!pip install torch==2.3.0 --index-url https://download.pytorch.org/whl/cpu
!pip uninstall -y scipy
!pip install scipy==1.11.4
!pip install torchtext
!pip install scikit-plot

#falta este
!pip install gensim



Found existing installation: torch 2.8.0+cu126
Uninstalling torch-2.8.0+cu126:
  Successfully uninstalled torch-2.8.0+cu126
Found existing installation: torchvision 0.23.0+cu126
Uninstalling torchvision-0.23.0+cu126:
  Successfully uninstalled torchvision-0.23.0+cu126
Found existing installation: torchaudio 2.8.0+cu126
Uninstalling torchaudio-2.8.0+cu126:
  Successfully uninstalled torchaudio-2.8.0+cu126
Looking in indexes: https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.4/190.4 MB 6.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
timm 1.0.22 requires torchvision, which is not installed.
fastai 2.8.5 requires torchvision>=0.11, which is not installed.
Found existing installation: scipy 1.16.3
Uninstalling scipy-1.16.3:
  Successfully uninstalled scipy-1.16.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 22.8 MB/s eta 0:00:00


In [7]:
import nltk
import numpy as np

from nltk import word_tokenize

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.base import BaseEstimator, TransformerMixin

# importe aquí sus clasificadores

import matplotlib.pyplot as plt

# word2vec
from gensim.models import Word2Vec, KeyedVectors
from gensim.models.phrases import Phrases, Phraser

# Pytorch imports
import torch
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

from torch.utils.data import DataLoader
from torchtext.data.functional import to_map_style_dataset

from torch import nn
from torch.nn import functional as F

from tqdm import tqdm
from sklearn.metrics import accuracy_score
import gc

from torch.optim import Adam
from torchtext.data import get_tokenizer



/usr/local/lib/python3.12/dist-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.12/dist-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


### Crear un clasificador basado en RNN

En esta parte de le pide definir un clasificador utilizando `PyTorch` con alguna arquitectura en Redes Recurrentes. Para ello debe realizar todos los pasos vistos en el tutorial 5, por lo que se recomienda revisarlo. Importante, no puede replicar ningún ejemplo de los del tutorial 5, debe proponer sus propias arquitecturas. Se le recomienda leer como utilizar variaciones de la RNN, como la LSTM o GRU en `Pytorch`.

Para completa esta parte deberá replicar el flujo de trabajo de como utilizar `PyTorch`. Esta esctrictamente prohibido utilizar variaciones que resuelvan directamente este problema, como `PyTorch Lightning` o `TensorFlow`. Los pasos a completar son los siguientes:

#### Cargar el dataset.

In [12]:
!wget https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/new/assignment_1/train/train.tsv

--2025-11-14 05:30:09--  https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/new/assignment_1/train/train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2080719 (2.0M) [text/plain]
Saving to: ‘train.tsv.2’

train.tsv.2         100%[===================>]   1.98M  --.-KB/s    in 0.04s   

2025-11-14 05:30:10 (48.8 MB/s) - ‘train.tsv.2’ saved [2080719/2080719]



In [16]:
# Dataset.
dataset_df = pd.read_csv("https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/new/assignment_1/train/train.tsv", sep="\t")
codes, unique_labels = pd.factorize(dataset_df['clase'])
dataset_df['clase_id'] = codes + 1 #Se asocia un valor numérico a la clase, lo necesitamos para el DataLoader
dataset_df.head()

,id,texto,clase,clase_id
0,12632,ultimo choro se 2018 que delicia,normal,1
1,7451,Pero es una realidad para muchas mujeres en Ve...,normal,1
2,4211,MALDITA SEAS COMUNA DE ÑUÑOA https://t.co/yN4E...,incivilidad,2
3,10199,Las tontas de #PautaLibre con el tremendo 🌶🌶 ...,incivilidad,2
4,11597,@user @user @user @user @user Devuelvete y and...,odio,3


In [21]:
target_classes

['normal', 'incivilidad', 'odio']

In [43]:
from sklearn.model_selection import train_test_split

#Hago 2 splits, para poder obtener data de entrenamiento, validación y test
#train_validation_df: Data de entrenamiento junto a data de validacion
#test_df: data para test
#train_df: data de entrenamiento obtenida de la division de train_validation_df
#val_df: data de validacion

train_validation_df, test_df = train_test_split(
    dataset_df,
    test_size=0.2,
    random_state=42,
    stratify=dataset_df["clase_id"] # Optional for balanced splits
)

train_df, val_df = train_test_split(
    train_validation_df,
    test_size=0.2,
    random_state=42,
    stratify=train_validation_df["clase_id"] # Optional for balanced splits
)


In [23]:
print(f"La data de entrenamiento tiene {len(train_df)} cantidad de datos")
print(f"La data de test tiene {len(test_df)} cantidad de datos")
print(f"La data de validacion tiene {len(val_df)} cantidad de datos")


La data de entrenamiento tiene 7816 cantidad de datos
La data de test tiene 2443 cantidad de datos
La data de validacion tiene 1955 cantidad de datos


In [24]:
#Ejemplos de datos
print(train_df.head(5))
print(test_df.head(5))
print(val_df.head(5))

         id                                              texto        clase  \
4906   7200  Leí a varios dando rt a los weones que empieza...  incivilidad   
3497   3124  Q se cree este enano maldito! Tenemos derecho ...  incivilidad   
6883   6985  Los Ctm de las extrema izquierda no respetan C...  incivilidad   
1773   1027  @user Tontona zurda y debe ser igual de ignora...  incivilidad   
10577  7758  Revisen el celular de sus hijos !\nSus contact...  incivilidad   

       clase_id  
4906          2  
3497          2  
6883          2  
1773          2  
10577         2  
         id                                              texto        clase  \
8623   1044  @user @user @user Hay que ver que tu si eres i...  incivilidad   
8335  12732  @user @user Ve al centro camina y nadie te con...         odio   
6258  10284  @user A los políticos les importa un rábano la...  incivilidad   
89     6021  gente? Acá lo del robinhoodismo es literalemnt...       normal   
3178  13576  La marico

In [44]:
#Función para convertir los dataframe anteriormente creados al nuevo dataset que necesitamos

def load_dataset(df: pd.DataFrame):
    for row in df.itertuples():
        yield int(row.clase_id), row.texto

In [51]:
train_dataset = load_dataset(train_df)
test_dataset = load_dataset(test_df)

#### Definir el vocabulario.

In [47]:
from torchtext.vocab import build_vocab_from_iterator

In [48]:
import re

def get_tokens(texto):
  for word in texto:
        clean_text = re.sub(r'[^A-Za-zñáéíóú]', ' ', word)
        tokens = clean_text.split()
  return tokens

In [49]:
def build_vocabulary(datasets):
    for dataset in datasets:
        for _, text in dataset:
            yield get_tokens(text)

vocab = build_vocab_from_iterator(build_vocabulary([train_dataset, test_dataset]), min_freq=1, specials=["<UNK>"])

vocab.set_default_index(vocab["<UNK>"])

#### Cargar el `DataLoader`

Recuerde que podría necesitar una función intermedia para procesar cada batch durante el entrenamiento, pero no es obligatorio hacerlo.

In [52]:
train_dataset_map_style, test_dataset_map_style = to_map_style_dataset(train_dataset), to_map_style_dataset(test_dataset)

max_words = 25

def vectorize_batch(batch):
    Y, X = list(zip(*batch))
    X = [vocab(get_tokens(text)) for text in X]
    X = [tokens+([0]* (max_words-len(tokens))) if len(tokens)<max_words else tokens[:max_words] for tokens in X]

    return torch.tensor(X, dtype=torch.int32), torch.tensor(Y) - 1

train_loader = DataLoader(train_dataset_map_style, batch_size=1024, collate_fn=vectorize_batch, shuffle=True)
test_loader  = DataLoader(test_dataset_map_style , batch_size=1024, collate_fn=vectorize_batch)

#### Definir la red recurrente.

Recuerde que debe difirnir los hyperparametros que estime conveniente.

In [70]:
import torch
import torch.nn as nn

embed_len = 128
hidden_dim = 256
n_layers = 2
num_classes = len(dataset_df["clase"].unique())
dropout_rate = 0.2

# --- LSTM ---
class RNNClassifier(nn.Module):
    def __init__(self):
        super(RNNClassifier, self).__init__()

        # 1. Embedding Layer: Converts token IDs into dense vectors
        self.embedding_layer = nn.Embedding(num_embeddings=len(vocab), embedding_dim=embed_len)

        # 2. LSTM Layer: The core recurrent layer.
        # dropout is applied to the output of each layer except the last.
        self.lstm = nn.LSTM(
            input_size=embed_len,
            hidden_size=hidden_dim,
            num_layers=n_layers,
            batch_first=True,
            dropout=dropout_rate
        )

        self.linear = nn.Linear(hidden_dim, num_classes)

        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

    def forward(self, X_batch):
        embeddings = self.embedding_layer(X_batch)
        batch_size = X_batch.size(0)
        device = X_batch.device

        h0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
        c0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)

        output, (hn, cn) = self.lstm(embeddings, (h0, c0))

        return self.linear(hn[-1])

#### Funciones de entrenamiento y evaluación.

Para esta parte, puede utilizar las funciones vista en el tutorial directamente. Pero es su reponsabilidad ajustarlas a su código.

In [71]:
def CalcValLossAndAccuracy(model, loss_fn, val_loader):
    with torch.no_grad():
        Y_shuffled, Y_preds, losses = [],[],[]
        for X, Y in val_loader:
            preds = model(X)
            loss = loss_fn(preds, Y)
            losses.append(loss.item())

            Y_shuffled.append(Y)
            Y_preds.append(preds.argmax(dim=-1))

        Y_shuffled = torch.cat(Y_shuffled)
        Y_preds = torch.cat(Y_preds)

        print("Valid Loss : {:.3f}".format(torch.tensor(losses).mean()))
        print("Valid Acc  : {:.3f}".format(accuracy_score(Y_shuffled.detach().numpy(), Y_preds.detach().numpy())))


def TrainModel(model, loss_fn, optimizer, train_loader, val_loader, epochs=10):
    for i in range(1, epochs+1):
        losses = []
        for X, Y in tqdm(train_loader):
            Y_preds = model(X)

            loss = loss_fn(Y_preds, Y)
            losses.append(loss.item())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print("Train Loss : {:.3f}".format(torch.tensor(losses).mean()))
        CalcValLossAndAccuracy(model, loss_fn, val_loader)

def MakePredictions(model, loader):
    Y_shuffled, Y_preds = [], []
    for X, Y in loader:
        preds = model(X)
        Y_preds.append(preds)
        Y_shuffled.append(Y)
    gc.collect()
    Y_preds, Y_shuffled = torch.cat(Y_preds), torch.cat(Y_shuffled)

    return Y_shuffled.detach().numpy(), F.softmax(Y_preds, dim=-1).argmax(dim=-1).detach().numpy()



##### Entrenamiento del modelo

Ejecute el entrenamiento de su modelo propuesto.

In [72]:
epochs = 5
learning_rate = 1e-3

loss_fn = nn.CrossEntropyLoss()
rnn_classifier = RNNClassifier()
optimizer = Adam(rnn_classifier.parameters(), lr=learning_rate)

TrainModel(rnn_classifier, loss_fn, optimizer, train_loader, test_loader, epochs)

100%|██████████| 8/8 [00:15<00:00,  1.96s/it]


Train Loss : 1.066
Valid Loss : 1.056
Valid Acc  : 0.444


100%|██████████| 8/8 [00:15<00:00,  1.96s/it]


Train Loss : 1.053
Valid Loss : 1.053
Valid Acc  : 0.444


100%|██████████| 8/8 [00:17<00:00,  2.22s/it]


Train Loss : 1.055
Valid Loss : 1.051
Valid Acc  : 0.444


100%|██████████| 8/8 [00:18<00:00,  2.30s/it]


Train Loss : 1.054
Valid Loss : 1.051
Valid Acc  : 0.444


100%|██████████| 8/8 [00:15<00:00,  1.99s/it]


Train Loss : 1.054
Valid Loss : 1.051
Valid Acc  : 0.444


##### Evaluacion del modelo

Ejecute la evaluación de su modelo, y genere un reporte de evaluación similar al de la pregunta anterior.

In [73]:
target_classes = ["normal", "incivilidad", "odio"] #[1,2,3]

In [74]:
Y_actual, Y_preds = MakePredictions(rnn_classifier, test_loader)

In [75]:
print("Test Accuracy : {}".format(accuracy_score(Y_actual, Y_preds)))
print("\nClassification Report : ")
print(classification_report(Y_actual, Y_preds, target_names=target_classes))
print("\nConfusion Matrix : ")
print(confusion_matrix(Y_actual, Y_preds))

Test Accuracy : 0.44412607449856734

Classification Report : 
              precision    recall  f1-score   support

      normal       0.00      0.00      0.00       856
 incivilidad       0.44      1.00      0.62      1085
        odio       0.00      0.00      0.00       502

    accuracy                           0.44      2443
   macro avg       0.15      0.33      0.21      2443
weighted avg       0.20      0.44      0.27      2443


Confusion Matrix : 
[[   0  856    0]
 [   0 1085    0]
 [   0  502    0]]


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Finalmente, análice sus resultados, ¿Porqué cree que obtuvo esos resultados?, ¿Es mejor que sólo utilizar Word Embeddings, porque?. Justique.

### Transformers BERT.

Para esta tarea se le piden crear una representación de texto usando la arquitectura basada en transformers, BERT:



In [76]:
!pip install transformers


#### Import BETO

Para esto debe importar el tokenizador y el modelo BETO.

In [2]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

#### Ejemplo de extracción de features.

In [78]:
# Uninstall the older version and then install the required modern version
!pip uninstall torch -y
!pip install --upgrade torch torchvision torchaudio

Found existing installation: torch 2.3.0+cpu
Uninstalling torch-2.3.0+cpu:
  Successfully uninstalled torch-2.3.0+cpu
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 136.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.3/322.3 MB 4.9 MB/s eta 0:00:00
   ━━━

Luego, debe cargar los modelos pre-entrenados:

In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased')
model = AutoModelForMaskedLM.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased', output_hidden_states=True).to(device)

Una vez cargado BETO, debe utilizarlo para extraer los embeddings asociados a la texto de su corpus. Se espera que usted realice lo siguiente:



Tokenizamos el texto para extraer los ids a cada palabra en el vocabulario interno de BETO, se recomienda utilizar el padding, trunctation, y max_length para considerar oraciones de diferentes tamaños.

Luego, debe verificar si cada uno de los ids extraídos se encuentran en la misma máquina donde fue cargado el modelo (CPU o GPU), se recomienda dejar todo en GPU.

In [4]:
# oración
sentence = "Hola, que tal? me gusta mucho el curso de NLP"

# extraemos los ids de los tokens, se recomienda definir los valores de las variables:
#  padding, truncation, max_length debido a que las secuencia de texto puede tener diferentes largos
inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors="pt", max_length=512)
# revisamos si cada de los ids, se encuentran en la misma máquina que el modelo (GPU o CPU)
inputs = {k: v.to(model.device) for k, v in inputs.items()}
inputs

{'input_ids': tensor([[    4,  1734,  1019,  1041,  1713,  1059,  1094,  2331,  1700,  1039,
           4430,  1009, 15567, 30968,     5]]),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

Una vez, extraídos los ids, usted debe obtener los estados ocultos de las ultimas capas de BETO.

In [5]:
# Extraemos la features
outputs = model(**inputs)

# ahora `outputs` tendrá el atributo `hidden_states`
hidden_states = outputs.hidden_states


Finalmente, debe guardar los embeddings en CPU los embeddings del token [CLS] que será usados en la clasificación del análisis de sentimientos.

In [6]:
import torch

In [7]:
with torch.no_grad():
# Seleccionamos la última capa y obtenemos el primer token ([CLS]) para cada ejemplo
# Estos son los embeddings que normalmente se usan para tareas de clasificación.
  cls_embeddings = hidden_states[-1][:, 0, :].cpu().numpy()


#### Extraer features de todo el dataset

Considerando lo anterior, usted debe implementar la función `get_beto_features_in_batches` para extraer los features de BETO (los estados ocultos y los embeddings del token [CLS]).

Esta función recibe dos parámetros, el texto a vectorizar y un tamaño de batch, debido a que es extramadamente recomendable a que procesen el texto por lotes, ya que si cargan todos el modelo se les agotará la memoria RAM.

In [14]:
#Si es necesario, cargamos la data nuevamente

import pandas as pd

!wget https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/new/assignment_1/train/train.tsv

dataset_df = pd.read_csv("https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/new/assignment_1/train/train.tsv", sep="\t")
codes, unique_labels = pd.factorize(dataset_df['clase'])
dataset_df['clase_id'] = codes + 1 #Se asocia un valor numérico a la clase, lo necesitamos para el DataLoader
dataset_df.head()

from sklearn.model_selection import train_test_split

train_validation_df, test_df = train_test_split(
    dataset_df,
    test_size=0.2,
    random_state=42,
    stratify=dataset_df["clase_id"] # Optional for balanced splits
)

train_df, val_df = train_test_split(
    train_validation_df,
    test_size=0.2,
    random_state=42,
    stratify=train_validation_df["clase_id"] # Optional for balanced splits
)


--2025-11-14 05:30:55--  https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/new/assignment_1/train/train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2080719 (2.0M) [text/plain]
Saving to: ‘train.tsv.3’

train.tsv.3         100%[===================>]   1.98M  --.-KB/s    in 0.04s   

2025-11-14 05:30:55 (53.7 MB/s) - ‘train.tsv.3’ saved [2080719/2080719]



In [9]:
from datasets import load_dataset, Dataset, DatasetDict

In [17]:
#Se crea un DatasetDict con la data de entrenamient, test y validación

train_dataset = Dataset.from_dict({
    'text': train_df['texto'].tolist(),
    'label': train_df['clase_id'].tolist()
})

validation_dataset = Dataset.from_dict({
    'text': val_df['texto'].tolist(),
    'label': val_df['clase_id'].tolist()
})

test_dataset = Dataset.from_dict({
    'text': test_df['texto'].tolist(),
    'label': test_df['clase_id'].tolist()
})

my_dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': validation_dataset,
    'test': test_dataset,
})

In [18]:
my_dataset_dict

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 7816
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1955
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2443
    })
})

In [31]:
def tokenize(batch):
  # Por cada batch vamos a obtener el texto, tokenizarlo y aplicar padding
  return tokenizer(batch["text"], padding=True, truncation=True, max_length=256)

def extract_hidden_states(batch):
 # Vamos a extraer los hidden states desde BERT, para esto pasamos el modelo
 # dispositivo
 inputs = {k:v.to(device) for k,v in batch.items() if k in tokenizer.model_input_names}
 # Extraemos solamente la last hidden state
 with torch.no_grad():
   # Changed model_distilbert to model_beto
   # Corrected: Access hidden_states tuple and take the last element
   last_hidden_state = model(**inputs).hidden_states[-1]
 # representaciones vectoriales que tanto queriamos :D
 return {"hidden_state": last_hidden_state[:,0].cpu().numpy()}

In [32]:
# Función para procesar los textos en lotes y obtener las características de BETO

BATCH_SIZE = 16

def get_beto_features_in_batches(texts, batch_size):
  texts_encoded = texts.map(tokenize, batched=True, batch_size=BATCH_SIZE)
  texts_encoded.set_format("torch", columns=["input_ids", "attention_mask", "label", "text"])
  texts_hidden = texts_encoded.map(extract_hidden_states, batched=True, batch_size=BATCH_SIZE)
  return texts_hidden

device


device(type='cuda')

Ahora extraíga los features, un punto importante es que la extracción de features podría tomar alrededor de una a dos horas dependiendo del tamaño del batch que utilicen.

In [33]:
train_embs = get_beto_features_in_batches(my_dataset_dict, BATCH_SIZE)

Map:   0%|          | 0/7816 [00:00<?, ? examples/s]

Map:   0%|          | 0/1955 [00:00<?, ? examples/s]

Map:   0%|          | 0/2443 [00:00<?, ? examples/s]

Map:   0%|          | 0/7816 [00:00<?, ? examples/s]

Map:   0%|          | 0/1955 [00:00<?, ? examples/s]

Map:   0%|          | 0/2443 [00:00<?, ? examples/s]

In [34]:
train_embs

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask', 'hidden_state'],
        num_rows: 7816
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask', 'hidden_state'],
        num_rows: 1955
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask', 'hidden_state'],
        num_rows: 2443
    })
})

#### Clasificación

Una vez extraído los features de BETO, realice la clasificación de los embeddings obtenidos. Debe implementar dos clasificadores a su elección.

In [44]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# Se extraen las features obtenidas con get_beto_features_in_batches para entrenamiento y test

X_train = np.array(train_embs['train']['hidden_state'])
y_train = np.array(train_embs['train']['label'])

X_test = np.array(train_embs['test']['hidden_state'])
y_test = np.array(train_embs['test']['label'])

print(f"Train Features Shape: {X_train.shape}")
print(f"Test Features Shape: {X_test.shape}")

Train Features Shape: (7816, 768)
Test Features Shape: (2443, 768)


In [45]:
#Se crean los 2 clasificadores

# Clasificador 1
clf1 = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(random_state=42, n_jobs=-1, n_estimators=100))
])

# Clasificador 2
clf2 = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(
      C=1.0,
      max_iter=1000,
      solver='liblinear',
      random_state=42
  ))
])

In [46]:
clf1.fit(X_train, y_train)


Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier',
                 RandomForestClassifier(n_jobs=-1, random_state=42))])

In [47]:
clf2.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier',
                 LogisticRegression(max_iter=1000, random_state=42,
                                    solver='liblinear'))])

#### Reporte de evaluación

Una vez realizada la clasificación, realice el reporte de clasificación y el análsis de la matriz confusión para ambos clasificadores.

Recuerde que para hacer esto, debe extraer los features del dataset de testing.

In [48]:
from sklearn.metrics import classification_report

In [49]:
#Reporte clasificador 1
y_pred = clf1.predict(X_test)
report = classification_report(y_test, y_pred)

print("\n--- Classification Report (Test Set) ---")
print(report)


--- Classification Report (Test Set) ---
              precision    recall  f1-score   support

           1       0.65      0.59      0.62       856
           2       0.61      0.85      0.71      1085
           3       0.88      0.22      0.36       502

    accuracy                           0.63      2443
   macro avg       0.71      0.56      0.56      2443
weighted avg       0.68      0.63      0.60      2443



In [50]:
#Reporte clasificador 2
y_pred = clf2.predict(X_test)
report = classification_report(y_test, y_pred)

print("\n--- Classification Report (Test Set) ---")
print(report)


--- Classification Report (Test Set) ---
              precision    recall  f1-score   support

           1       0.66      0.65      0.65       856
           2       0.69      0.74      0.71      1085
           3       0.59      0.51      0.55       502

    accuracy                           0.66      2443
   macro avg       0.65      0.63      0.64      2443
weighted avg       0.66      0.66      0.66      2443



Finalmente, que puede decir de los resultados obtenidos. ¿Se diferencia de los resultados obtenidos de la red RNN? ¿A que se debe esto? Justifique

### Large Language Model

##### Zero Shot Learning

Utilizando la técnica de zero shot learning, utilice la API de `openai` para clasificar el texto del dataset.

Además, genere el reporte de clasificación usando `scikit-learn` como en las preguntas anteriores.

Recuerde solicitar al profesor auxiliar el TOKEN para hacer consultas al LLM.

##### Few Shot Learning

Utilizando la técnica de few shot learning, utilice la API de `openai` para clasificar el texto del dataset.

Además, genere el reporte de clasificación usando `scikit-learn` como en las preguntas anteriores.

Recuerde solicitar al profesor auxiliar el TOKEN para hacer consultas al LLM.